In [79]:
radar = pyart.io.read('KDDC20160524_181120_V06.gz')
radar = pyart.io.read('KDDC20160525_001527_V06.gz')

field = 'velocity'
field = 'reflectivity'

field = 'differential_phase'
field = "spectrum_width"
field = 'differential_reflectivity'
field = 'cross_correlation_ratio'

field = 'reflectivity'
field = 'velocity'

In [2]:
display = pyart.graph.RadarDisplay(radar)

In [69]:
radar.metadata 

{'Conventions': 'CF/Radial instrument_parameters',
 'comment': '',
 'history': '',
 'institution': '',
 'instrument_name': '',
 'original_container': 'NEXRAD Level II',
 'references': '',
 'source': '',
 'title': '',
 'vcp_pattern': 32,
 'version': '1.3'}

In [97]:
LookUp = []
iter_obj = radar.iter_elevation()
for n, elev in enumerate(iter_obj):
    if elev.shape[0] == 720:
        if np.mod(n,2) == 0:
            LookUp.append((n, n+1))
    else:
        LookUp.append((n,n))
   
print LookUp

[(0, 1), (2, 3), (4, 5), (6, 7), (8, 8), (9, 9), (10, 10), (12, 13), (13, 13), (14, 14), (15, 15), (16, 16), (18, 19), (19, 19), (20, 20), (21, 21), (22, 22)]


In [109]:
for n, m in LookUp:
    begin, end = radar.get_start_end(n)
    dbz = radar.get_field(n, 'reflectivity')
    vel = radar.get_field(m, 'velocity')
    print("DBZ index:  %d   VEL index:  %d" % (n,m))
    print("REF  :  %d" % (np.sum(dbz.mask == False)))
    print("VEL  :  %d" % (np.sum(vel.mask == False)))



DBZ index:  0   VEL index:  1
REF  :  332431
VEL  :  251063
DBZ index:  2   VEL index:  3
REF  :  315240
VEL  :  253457
DBZ index:  4   VEL index:  5
REF  :  305960
VEL  :  262746
DBZ index:  6   VEL index:  7
REF  :  331457
VEL  :  253348
DBZ index:  8   VEL index:  8
REF  :  148366
VEL  :  113379
DBZ index:  9   VEL index:  9
REF  :  154247
VEL  :  112494
DBZ index:  10   VEL index:  10
REF  :  153524
VEL  :  115951
DBZ index:  12   VEL index:  13
REF  :  278332
VEL  :  123733
DBZ index:  13   VEL index:  13
REF  :  147968
VEL  :  123733
DBZ index:  14   VEL index:  14
REF  :  139942
VEL  :  129418
DBZ index:  15   VEL index:  15
REF  :  126312
VEL  :  124146
DBZ index:  16   VEL index:  16
REF  :  111502
VEL  :  111500
DBZ index:  18   VEL index:  19
REF  :  275815
VEL  :  98035
DBZ index:  19   VEL index:  19
REF  :  98039
VEL  :  98035
DBZ index:  20   VEL index:  20
REF  :  82441
VEL  :  82438
DBZ index:  21   VEL index:  21
REF  :  69319
VEL  :  69316
DBZ index:  22   VEL index:

In [105]:
for n, sweep in enumerate(radar.iter_field(field)):
    begin, end = radar.get_start_end(n)
    print radar.elevation['data'][begin:end].mean()
    print("%s  :  %f  %f" % (field, sweep.data[n].max(), sweep.data[n].min()))
    print("%s  :  %d" % (field, np.sum(sweep.mask[n] == True)))
    print("%s  :  %d" % (field, np.sum(sweep.mask[n] == False)))


0.455413
velocity  :  -64.000000  -64.000000
velocity  :  1832
velocity  :  0
0.483398
velocity  :  26.000000  -64.500000
velocity  :  1096
velocity  :  736
0.850569
velocity  :  -64.000000  -64.000000
velocity  :  1832
velocity  :  0
0.878906
velocity  :  16.500000  -64.500000
velocity  :  1448
velocity  :  384
1.29058
velocity  :  -64.000000  -64.000000
velocity  :  1832
velocity  :  0
1.31836
velocity  :  10.500000  -64.500000
velocity  :  1487
velocity  :  345
0.456246
velocity  :  -64.000000  -64.000000
velocity  :  1832
velocity  :  0
0.483398
velocity  :  32.000000  -64.500000
velocity  :  1531
velocity  :  301
1.77139
velocity  :  -5.500000  -64.500000
velocity  :  1672
velocity  :  160
2.38911
velocity  :  4.000000  -64.500000
velocity  :  1512
velocity  :  320
3.10626
velocity  :  -2.500000  -64.500000
velocity  :  1431
velocity  :  401
0.486114
velocity  :  -64.000000  -64.000000
velocity  :  1832
velocity  :  0
0.463156
velocity  :  23.500000  -64.500000
velocity  :  1487
v

In [106]:
radar.get_field?